In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

from src.extractor_utils import (predict, getPrediction,features_saving,features_loading)

from utils.utils import (create_submission ,metrics2csv, save_results)

# Checking for resources avaliables

In [2]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         91G         47G         10G         81G        116G
Swap:            0B          0B          0B
Sun Nov 26 17:01:52 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   65C    P0   130W / 149W |  10770MiB / 11439MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

In [3]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [4]:
use_gpu = False
use_DataParalel= False 
use_CPU= True        

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using 24 CPU's


# Choose dataset 

In [24]:
path2train = "/mnt/home/e209440/data/train" 
path2test = "/mnt/home/e209440/data/test" 

In [25]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}   

In [26]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [27]:
dsets = {
    'train': KaggleSafeDriverDataset(path2train, transforms=data_transforms['train'],use_only=use_only),
    'valid': KaggleSafeDriverDataset(path2train, transforms=data_transforms['valid'],use_only=use_only, is_val=True, val_size=0.2),
    'test':  KaggleSafeDriverDataset(path2test, transforms=data_transforms['valid'],use_only=use_only, is_test=True),
}


# Path to features

In [5]:
path2features= "/mnt/home/e209440/state-farm-distracted-driver-detection/features/" 
#path2features= "/mnt/home/r120084/state-farm-distracted-driver-detection/features/" 

# Choosing Model

In [6]:
use_resnet = True
use_inception = False
use_denseNet = False

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()

Using ResNet model


In [7]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model.mrnc = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
        model.mrnc = model.mrnc.cuda()
        model.mrnd = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
        model.mrnd = model.mrnd.cuda()
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
else:
    print("Using CPU's")

Using CPU's


# Loading features

In [8]:
model_name

'ResNet'

In [9]:
load_feat= features_loading(path2features,model_name)

Loaded features with shapes: 


train:
pred torch.Size([17940, 6400]), true torch.Size([17940])

valid:
pred torch.Size([4484, 6400]), true torch.Size([4484])

test:
pred torch.Size([79726, 6400]), true torch.Size([79726])


In [10]:
load_feat['train'][0].shape,load_feat['train'][1].shape

(torch.Size([17940, 6400]), torch.Size([17940]))

In [11]:
conv_dset ={
'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
}

# Creating Dataloaders

In [12]:
batch_size= 32

dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [13]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

{'test': 2492, 'train': 561, 'valid': 141}

# Trainning Model

In [14]:
path2saveModel = '/mnt/home/r120084/project/models/'+model_name+'distractdriver'
#path2saveModel = '/mnt/home/e209440/models/'+model_name+'DistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=1)

In [16]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 50

optimizer =  optim.Adam(model.mrnd.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

params = {'model' : model.mrnd, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric(), ptt.PrintCallback()] #ptt.PlotCallback(),
}

In [17]:
trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,**params)

In [18]:
trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

Start training for 50 epochs
  1:  38.2s   T: 1.57844 0.43344   V: 1.05204 0.56735 best
  2:  54.9s   T: 0.42543 0.85167   V: 0.24852 0.92061 best
  3:  42.9s   T: 0.21593 0.93032   V: 0.12868 0.96164 best
  4:  36.6s   T: 0.16051 0.95050   V: 0.30979 0.89942 
  5:  37.0s   T: 0.12022 0.95987   V: 0.14393 0.95874 
  6:  40.6s   T: 0.14118 0.95658   V: 0.06958 0.97859 best
  7:  37.3s   T: 0.13501 0.95808   V: 0.10213 0.96521 
  8:  38.1s   T: 0.09632 0.96918   V: 0.17467 0.94068 
Best model was saved at epoch 6 with loss 0.06958: /mnt/home/r120084/project/models/ResNetdistractdriver
Stop training at epoch: 8/50


# Loading best epoch

In [19]:
trainer.load_state(path2saveModel)
model.mrnd = trainer.model

In [46]:
metrics2csv(trainer, model_name)

done!


In [21]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

evaluate: 560/560 ok
evaluate: 140/140 ok


In [22]:
train_eval, valid_eval

({'losses': 0.04498275536483148}, {'losses': 0.06958254571110144})

In [28]:
# need to create a dataloader without shuffle to compare the mismatch
dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= False)

In [38]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_convnet['train'], model.mrnd, use_gpu=use_gpu)
result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)
result_test = predict(dset_loaders_convnet['test'], model.mrnd, use_gpu=use_gpu)

predict: 560/560 ok
predict: 140/140 ok
predict: 2491/2491 ok


In [50]:
predictions_out = {'train': result_train, 'valid': result_valid, 'test': result_test}

In [53]:
save_results(predictions_out, model_name)

train set result saved!
valid set result saved!
test set result saved!


In [62]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [63]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [64]:
print('Train: ', correct_train, '/', len(dsets['train']) )
print('Valid: ', correct_valid, '/', len(dsets['valid']) )

Train:  17926 / 17940
Valid:  4462 / 4484


# Make submission of the Test set

In [45]:
create_submission(result_test, model_name + '_distracted_driver')

done!
